<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Fixa%C3%A7%C3%A3o_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [1]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("/content/winemag-data_first150k.csv") \
  .show()

+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|_c0|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley

In [17]:
spark

# Introdução
Para as questões quem envolverem manipulação de Dataframes, utilize o dataset winemag-data_first150k.csv:

https://www.kaggle.com/zynicide/wine-reviews.

Faremos algumas análises utilizando esse dataset.

In [24]:
csv_path = "winemag-data_first150k.csv"

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("description", StringType(), True),
    StructField("designation", StringType(), True),
    StructField("points", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("province", StringType(), True),
    StructField("region_1", StringType(), True),
    StructField("region_2", StringType(), True),
    StructField("variety", StringType(), True),
    StructField("winery", StringType(), True)
])

df = spark.read.csv(path=csv_path, header="true", schema=schema)
df.show()

+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
| id|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley

# Questão 1
Escreva um conjunto de transformações em Pyspark API e SparkSQL seguido de uma ação `.show()` para exibir a **média das notas (points) obtidos por cada país (country), ordenado decrescentemente pelas notas**:


In [25]:
# SparkSQL
df.createOrReplaceView("wines")

spark.sql("""

SELECT 
  country,
  AVG(points) as Media
  FROM wines
  GROUP by country
  Media DESC

""").show()


AttributeError: ignored

In [8]:
# Pyspark
import pyspark.sql.functions as f
df.select("country", "points") \
  .groupBy("country") \
  .agg(
      f.avg("points").desc()) \
  .show()
  

NameError: ignored

# Questão 2
Escreva um conjunto de transformações em Pyspark API e SparkSQL seguido de uma ação `.show()` para exibir as **top 20 vinícola (campo winery) mais caras, juntamente com o país e a província**:

In [26]:
# SparkSQL

df.createOrReplaceView("wines")

spark.sql("""

SELECT 
    country,
    province,
    winery,
    MAX(price) as maior_preco
  FROM wines
  GROUP by country
  ORDER BY maior_preco DESC

""").show()

AttributeError: ignored

In [ ]:
# Pyspark
df.select("country","province", "winery", "price") \
  .groupBy

# Questão 3
Escreva um conjunto de transformações em Pyspark API e SparkSQL seguido de uma ação `.show()` para exibir as **a média de preços para cada faixa de pontuação de 10 em 10**:

ex: 
```
| faixa | media_preço |
-----------------------
|10 ~ 20|      19     |
|20 ~ 30|      23     |
|...    |      ..     |
|90 ~100|      95     |
```

In [27]:
(11) -> ('10 ~ 20')
(17) -> ('10 ~ 20')
(89) -> ('80 ~ 90')
(97) -> ('90 ~ 100')

(11) -> ('10')
(17) -> ('10')
(89) -> ('80')
(97) -> ('90')

selectExpr("concat(cast(points as string)[0], 0)")


# SparkSQL

SyntaxError: ignored

In [ ]:
# Pyspark
def get_range(value):
  return str(int(number/10)) + '0' + 


  df.createGlobalTempView("wines")
  

# Questão 4

Utilize os RDDs para realizar a contagem de cada letra dado o texto abaixo, ou seja, ao final, teremos a seguinte estrutura:
```
('a', 35)
('b', 32)
('c' 5)
...
```

In [ ]:
text = "At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures. A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only added to, such as counters and sums. This guide shows each of these features in each of Spark’s supported languages. It is easiest to follow along with if you launch Spark’s interactive shell – either bin/spark-shell for the Scala shell or bin/pyspark for the Python one."

In [ ]:
# ...

# Questão 5

Imagine que você foi contratado como engenheiro de dados para construir uma aplicação que seja capaz de processar dados em near real-time relativos a transações bancárias em um banco X.

Basicamente, as trasações ocorrem enquanto o usuário utiliza o app do banco X. A cada transação, esses dados são enviados para um servidor em nuvém que irá recebê-los e encaminhar para o sistema de streaming classificá-los.

Esse streaming servirá de base para que uma equipe anti-fraude, do banco X, seja informada sempre que uma transação suspeita ocorrer e tomará ações o mais rápido o possível. Assim sendo, o **streaming não será ativo, ou seja, não tomará ações. Ele apenas informará em real-time sobre uma possível fraude em algum lugar**.

**BONUS**: Em paralelo a isso, para que seja possível gerar um streaming preciso, um modelo de machine learning é alimentado diariamente com os históricos dos últimos 5 anos das transações bancárias, consideradas fraudes ou não. Esse modelo é treinado diariamente e o modelo gerado é disponibilizado para o streaming, sempre que possível.

Como você desenharia a arquitetura dessa solução? Quais os melhores componentes? Como seria o seu pipeline de dados?

Descreva utilizando um desenho. Pode mandar uma foto do desenho (a caneta) ou usar uma ferramenta web para desenhar a arquitura (draw.io). Favor enviar diretamente para o professor no Slack.

# Questão 6

Descreva o que são **accumulators** e **broadcast variables** e explique quando e porque utilizá-las.


# Questão 7

Utilizando o dado proveniente do Kafka, já visto em sala de aula, implemente um contador de transações em streaming para cada **conta destino** (campo target). Crie duas versões do streaming:

- utilizando Structured Streaming
- utilizando Spark Streaming

Ex.:
```
# target, coutn
('aZ', 123)
('as', 43)
```

In [ ]:
# Structured Streaming

In [ ]:
# Spark Streaming